# RO2
1. Examining factors that influence working over the required working hours and under the required working hours.

**2. Conducting association rule analysis to identify factors influencing the mismatch between highest qualifications and employment qualifications.**

3. Investigating how individuals address the skill gap when they lack the necessary qualifications in their current profession

In [1]:
import pandas  as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Prepare data

In [65]:
df = pd.read_csv('data_sorted.csv')
codebook = pd.read_excel('PIAAC\Cycle 1\International Codebook_PIAAC Public-use File (PUF) Variables and Values.xlsx')

print(df.shape)
df.head()

<>:2: DeprecationWarning: invalid escape sequence '\C'
<>:2: DeprecationWarning: invalid escape sequence '\C'
C:\Users\Anis\AppData\Local\Temp\ipykernel_35404\2046365629.py:2: DeprecationWarning: invalid escape sequence '\C'
  codebook = pd.read_excel('PIAAC\Cycle 1\International Codebook_PIAAC Public-use File (PUF) Variables and Values.xlsx')
C:\Users\Anis\AppData\Local\Temp\ipykernel_35404\2046365629.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_sorted.csv')


(11470, 93)


,Skill use work - ICT - Computer - How often - Programming language,ISCO1C,Skill use work - ICT - Computer - How often - Real-time discussions,Skill use work - How often - Organising own time,Skill use everyday life - ICT - Computer - How often - Word,Skill use work - How often - Planning others activities,PVPSL6,About yourself - Learning strategies - Attribute something new,ICTHOME,PVPSL8,...,Education - Highest qualification - Level,PVPSL4,PVPSL10,Skill use everyday life - ICT - Internet - How often - In order to better understand various issues,About yourself - Learning strategies - Get to the bottom of difficult things,WorkHours,WorkHours_Difference,Qualification_Comparision,Qualification_Status,PVPSL_Performance
0,At least once a week but not every day,Service workers and shop and market sales workers,Less than once a month,Every day,Less than once a week but at least once a month,Never,315.18911,To a high extent,2.021940,296.26412,...,"ISCED 5A, bachelor degree",342.29315,318.98045,Less than once a week but at least once a month,To a high extent,Over Average,18,Over by 5 level(s),Over,Moderate performer
1,Less than once a week but at least once a month,Professionals,Never,Every day,Never,Less than once a month,326.64999,To some extent,1.646234,273.53912,...,"ISCED 5A, bachelor degree",325.28718,296.98809,At least once a week but not every day,To a high extent,Over Average,8,Equal,Equal,Moderate performer
2,Never,"Legislators, senior officials and managers",At least once a week but not every day,Every day,At least once a week but not every day,At least once a week but not every day,285.41693,To some extent,3.430657,279.23163,...,"ISCED 5A, bachelor degree",316.26258,249.59748,Every day,Very little,Over Average,18,Under by 2 level(s),Under,Weak performer
3,Never,Service workers and shop and market sales workers,Never,Less than once a month,Less than once a month,Never,332.55340,To a high extent,1.623673,284.82134,...,ISCED 5B,318.41257,325.03424,At least once a week but not every day,To some extent,Under Average,-15,Over by 5 level(s),Over,Moderate performer
4,Never,Technicians and associate professionals,Never,Never,Never,Never,300.41854,To some extent,1.365424,290.70099,...,ISCED 3C 2 years or more,307.69612,307.37447,Less than once a month,Very little,Under Average,-10,Over by 1 level(s),Over,Moderate performer


In [66]:
# variables

# dropped AGE_R, ISCO2C
demographic_var = ['CNTRYID', 'GENDER_R', 'LNG_HOME', 'NATIVELANG', 'NATIVESPEAKER', 'J_Q06b', 'J_Q07b', 'I_Q08', 'J_Q04a']
eduwork_var = ['CNTRYID', 'C_Q07', 'C_D06', 'B_Q01a', 'B_Q01a3', 'B_Q01b', 'D_Q12a', 'B_Q05c', 'B_Q10c', 'D_Q14', 'ISCOSKIL4', 'ISCO1C', 'D_Q10', 'D_Q03', 'D_Q04', 'D_Q07a', 'D_Q07b', 'D_Q08a', 'D_Q08b', 'VET', 'NFEHRS']

worksl_var = ['CNTRYID', 'ICTWORK', 'NFEHRSJR']
worksl_var.extend(codebook.loc[codebook['Name'].str.startswith('F_'), 'Name'].tolist())
worksl_var.extend(codebook.loc[codebook['Name'].str.startswith('G_') & codebook['Label'].str.contains('ICT'), 'Name'].tolist())

lifesl_var = ['CNTRYID', 'ICTHOME', 'NFEHRSNJR']
lifesl_var.extend(codebook.loc[codebook['Name'].str.startswith('H_') & codebook['Label'].str.contains('ICT'), 'Name'].tolist())
lifesl_var.extend(codebook.loc[codebook['Name'].str.startswith('I_') & codebook['Label'].str.contains('Learning'), 'Name'].tolist())

pvpsl = []
pvpsl.extend(codebook.loc[codebook['Name'].str.startswith('PVPSL'), 'Name'].tolist())


vars = list(set(demographic_var + eduwork_var + worksl_var + lifesl_var + pvpsl))
vars_keep = [var for var in vars if not (var[0].isalpha() and var[1] == '_')]
vars_change = [var for var in vars if var[0].isalpha() and var[1] == '_']

vars_dict = {}

for i in vars_change:
    match = codebook.loc[codebook['Name'] == i, 'Label'].tolist()
    if match:
        vars_dict[i] = match[0]


demographic_mapped = [vars_dict.get(value, value) if value is not None else None for value in demographic_var]
eduwork_mapped = [vars_dict.get(value, value) if value is not None else None for value in eduwork_var]
worksl_mapped = [vars_dict.get(value, value) if value is not None else None for value in worksl_var]
lifesl_mapped = [vars_dict.get(value, value) if value is not None else None for value in lifesl_var]

hours_var = ['WorkHours', 'WorkHours_Difference']
match_var = ['Qualification_Comparision', 'Qualification_Status']
performance_var = 'PVPSL_Performance'

print(len(demographic_mapped) + len(eduwork_mapped) + len(worksl_mapped) + len(lifesl_mapped) + len(hours_var) + len(match_var) + len(performance_var))

100


# Data Setup

In [67]:
from sklearn.impute import SimpleImputer

In [68]:
count_per_column = df.apply(lambda col: col[col == 'Not stated or inferred'].count())
notstated_cols = []

for col, count in count_per_column.items():
    if count > 0:
        print('Column: ', col, '\tCount:', count)
        notstated_cols.append(col)

Column:  Skill use work - ICT - Computer - How often - Programming language 	Count: 2442
Column:  ISCO1C 	Count: 32
Column:  Skill use work - ICT - Computer - How often - Real-time discussions 	Count: 2441
Column:  Skill use work - How often - Organising own time 	Count: 11
Column:  Skill use everyday life - ICT - Computer - How often - Word 	Count: 1144
Column:  Skill use work - How often - Planning others activities 	Count: 17
Column:  About yourself - Learning strategies - Attribute something new 	Count: 11
Column:  Skill use work - ICT - Computer - How often - Spreadsheets 	Count: 2441
Column:  NATIVELANG 	Count: 603
Column:  Skill use work - How often - Influencing people 	Count: 17
Column:  Skill use work - ICT - Computer - Lack of skills affect career 	Count: 2458
Column:  Current work - Requirements - Education level 	Count: 1649
Column:  Education - Highest qualification - Level of foreign qualification 	Count: 11452
Column:  Skill use everyday life - ICT - Internet - How ofte

In [69]:
for index, row in df.iterrows():
    foreign = row['Education - Highest qualification - Level of foreign qualification']
    edu = row['Education - Highest qualification - Level']
    
    if edu == 'Not stated or inferred':
        df.at[index, 'Education - Highest qualification - Level'] = foreign

df.drop(columns='Education - Highest qualification - Level of foreign qualification', inplace=True)

c:\Users\Anis\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1429: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [70]:
jp = df[df['CNTRYID'] == 'Japan'].copy()
sg = df[df['CNTRYID'] == 'Singapore'].copy()
kz = df[df['CNTRYID'] == 'Kazakhstan'].copy()
kr = df[df['CNTRYID'] == 'Korea'].copy()

countries = [jp, sg, kz, kr]

In [71]:
imputer = SimpleImputer(strategy='most_frequent')

for current in countries:
    count_per_column = df.apply(lambda col: col[col == 'Not stated or inferred'].count())
    notstated_cols = []
    
    for col, count in count_per_column.items():
        if (current[col] != 'Not stated or inferred').any():
            notstated_cols.append(col)

    current.replace('Not stated or inferred', np.nan, inplace=True)
    current[notstated_cols] = imputer.fit_transform(current[notstated_cols])

c:\Users\Anis\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1429: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\Anis\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1429: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\Anis\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1429: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.h

In [72]:
df = pd.concat([jp, sg, kz, kr])

nan_col = []

print('NULLS:')
for col in df.columns:
    if df[col].isna().sum() > 0:
        print(col)
        nan_col.append(col)

df[nan_col] = imputer.fit_transform(df[nan_col])

print('STILL NULL:')
for col in df.columns:
    if df[col].isna().sum() > 0:
        print(col)

NULLS:
LNG_HOME
AGE_R
STILL NULL:


In [73]:
sns.set_palette("dark")
sns.set_style('darkgrid')
sns.set(font_scale=1.1)

In [74]:
print(df.shape)
df['Qualification_Status'].value_counts()

(11470, 92)


Qualification_Status
Equal    4966
Over     4834
Under    1670
Name: count, dtype: int64

In [75]:
drop = pvpsl
drop.append('Qualification_Comparision')

# df2 = df[df['Qualification_Status'] != 'Equal']
df2 = df.drop(columns=drop)

display(df2['Qualification_Status'].value_counts())
display(df2.shape)

Qualification_Status
Equal    4966
Over     4834
Under    1670
Name: count, dtype: int64

(11470, 81)

In [76]:
over_df = df2[df2['Qualification_Status'] == 'Over']
under_df = df2[df2['Qualification_Status'] == 'Under']
equal_df = df2[df2['Qualification_Status'] == 'Equal']

print(over_df.shape)
print(under_df.shape)
print(equal_df.shape)

(4834, 81)
(1670, 81)
(4966, 81)


# ARM with MlExtend

In [77]:
from apyori import apriori
import mlxtend.frequent_patterns as fp

results_path = "RQ2/"

In [78]:
target = 'Qualification_Status'

# get only the 1 cons

def refineRules(type, assoc_rules, support, threshold):
    filename_con_all = results_path + type + '/ARM_consequents_' + str(support) + '_' + str(threshold) + '.csv'
    filename_con = results_path + type + '/ARM_consequents_ONE_' + str(support) + '_' + str(threshold) + '.csv'

    con_all_condition = assoc_rules['consequents'].apply(lambda x: any(target in str(item) for item in x))
    con_one_condition = assoc_rules['consequents'].apply(lambda x: any(target in str(item) for item in x) and len(x) == 1)

    consequents_all = assoc_rules[con_all_condition]
    consequents_one = assoc_rules[con_one_condition]
    
    print(len(consequents_all))
    print(len(consequents_one))

    consequents_all.to_csv(filename_con_all, index=False)
    consequents_one.to_csv(filename_con, index=False)

In [79]:
def makeAssocRules(type, data, support, threshold):
    # filename_growth = results_path + type + '/ARM_Growth_' + str(support) + '.csv'

    fp_apriori = fp.apriori(data,min_support=support,use_colnames=True)
    # fp_fpgrowth = fp.fpgrowth(data, min_support=support, use_colnames=True)
    
    assoc_rules = fp.association_rules(fp_apriori,metric='confidence', min_threshold=threshold,support_only=False)
    # fp_fpgrowth.to_csv(filename_growth, index=False)
    
    refineRules(type, assoc_rules, support, threshold)

In [80]:
# Over
over_onehot = pd.get_dummies(over_df, columns=over_df.columns)
display(over_onehot.head(5))
display(over_onehot.shape)

# Under
under_onehot = pd.get_dummies(under_df, columns=under_df.columns)
display(under_onehot.head(5))
display(under_onehot.shape)

# Equal
equal_onehot = pd.get_dummies(equal_df, columns=equal_df.columns)
display(equal_onehot.head(5))
display(equal_onehot.shape)


,Skill use work - ICT - Computer - How often - Programming language_At least once a week but not every day,Skill use work - ICT - Computer - How often - Programming language_Every day,Skill use work - ICT - Computer - How often - Programming language_Less than once a month,Skill use work - ICT - Computer - How often - Programming language_Less than once a week but at least once a month,Skill use work - ICT - Computer - How often - Programming language_Never,ISCO1C_Armed forces,ISCO1C_Clerks,ISCO1C_Craft and related trades workers,ISCO1C_Don't know,ISCO1C_Elementary occupations,...,WorkHours_Difference_52,WorkHours_Difference_58,WorkHours_Difference_63,WorkHours_Difference_64,WorkHours_Difference_70,Qualification_Status_Over,PVPSL_Performance_At risk,PVPSL_Performance_Moderate performer,PVPSL_Performance_Strong performer,PVPSL_Performance_Weak performer
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
6,False,False,False,False,True,False,False,True,False,False,...,False,False,False,False,False,True,True,False,False,False
9,False,False,False,False,True,False,True,False,False,False,...,False,False,False,False,False,True,False,False,True,False


(4834, 4998)

,Skill use work - ICT - Computer - How often - Programming language_At least once a week but not every day,Skill use work - ICT - Computer - How often - Programming language_Every day,Skill use work - ICT - Computer - How often - Programming language_Less than once a month,Skill use work - ICT - Computer - How often - Programming language_Less than once a week but at least once a month,Skill use work - ICT - Computer - How often - Programming language_Never,ISCO1C_Armed forces,ISCO1C_Clerks,ISCO1C_Craft and related trades workers,ISCO1C_Don't know,ISCO1C_Elementary occupations,...,WorkHours_Difference_29,WorkHours_Difference_32,WorkHours_Difference_36,WorkHours_Difference_42,WorkHours_Difference_58,Qualification_Status_Under,PVPSL_Performance_At risk,PVPSL_Performance_Moderate performer,PVPSL_Performance_Strong performer,PVPSL_Performance_Weak performer
2,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
12,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
13,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,True,True,False,False,False
18,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
20,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


(1670, 2779)

,Skill use work - ICT - Computer - How often - Programming language_At least once a week but not every day,Skill use work - ICT - Computer - How often - Programming language_Every day,Skill use work - ICT - Computer - How often - Programming language_Less than once a month,Skill use work - ICT - Computer - How often - Programming language_Less than once a week but at least once a month,Skill use work - ICT - Computer - How often - Programming language_Never,ISCO1C_Armed forces,ISCO1C_Clerks,ISCO1C_Craft and related trades workers,ISCO1C_Don't know,ISCO1C_Elementary occupations,...,WorkHours_Difference_47,WorkHours_Difference_48,WorkHours_Difference_52,WorkHours_Difference_53,WorkHours_Difference_58,Qualification_Status_Equal,PVPSL_Performance_At risk,PVPSL_Performance_Moderate performer,PVPSL_Performance_Strong performer,PVPSL_Performance_Weak performer
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
5,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
7,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
8,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
11,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False


(4966, 5145)

In [81]:
support = 0.5
thresh_range = 1
threshold = 0.9

In [82]:
makeAssocRules('Over', over_onehot, support, threshold)
print('Over = COMPLETED')

403204
39419
Over = COMPLETED


In [83]:
makeAssocRules('Under', under_onehot, support, threshold)
print('Under = COMPLETED')

638332
50167
Under = COMPLETED


In [84]:
makeAssocRules('Equal', equal_onehot, support, threshold)
print('Equal = COMPLETED')

797219
56927
Equal = COMPLETED


In [14]:
# for i in range(supp_range, 10, 1):
#     support = i / 10
    
#     # for j in range(thresh_range, 10, 1):
#         # threshold = j / 10

#     makeAssocRules('Over', over_onehot, support, threshold)

In [15]:
# for i in range(supp_range, 10, 1):
#     support = i / 10
    
#     # for j in range(thresh_range, 10, 1):
#     #     threshold = j / 10
        
#     makeAssocRules('Under', under_onehot, support, threshold)
#     print('Support : ', support, ' [COMPLETED]')

In [16]:
# for i in range(supp_range, 10, 1):
#     support = i / 10
    
#     # for j in range(thresh_range, 10, 1):
#     #     threshold = j / 10
        
#     makeAssocRules('Equal', equal_onehot, support, threshold)
#     print('Support : ', support, ' [COMPLETED]')

# Looking at the rules

In [139]:
def clean_string(s):
    # Remove the 'frozenset({' prefix and '})' suffix
    cleaned = s.replace("frozenset({", "").replace("})", "")
    
    # Remove the leading and trailing single quotes and split by ', '
    elements = [elem.strip().strip("'") for elem in cleaned.split(", ")]
    
    # Join the elements back into a single string, separated by commas
    return ', '.join(elements)

In [140]:
def cleanStrings(data):
    data['antecedents'] = [clean_string(s) for s in data['antecedents']]
    data['consequents'] = [clean_string(s) for s in data['consequents']]
    
    return data

In [141]:
def pruneRules(data, lift = 1, confidence = 0.9, support = 0.9):
    newData = data.copy()
    
    newData = newData[newData['lift'] > lift]
    newData = newData[newData['confidence'] > confidence]
    newData = newData[newData['support'] > support]
    
    newData.sort_values(by='lift', ascending=False, inplace=True)
    newData.reset_index(drop = True, inplace = True)

    print(newData.shape)
    print('Range for LIFT: ', np.min(newData['lift']), ' - ', np.max(newData['lift'])) 
    print('Range for CONFIDENCE: ', np.min(newData['confidence']), ' - ', np.max(newData['confidence'])) 
    print('Range for SUPPORT: ', np.min(newData['support']), ' - ', np.max(newData['support'])) 

    return newData

In [142]:
def printUnique(data):
    print('antecedents')
    print(len(data['antecedents'].unique()))

    for i in data['antecedents'].unique():
        print(i)
    
    print('-'*150)
    
    print('consequents')
    print(len(data['consequents'].unique()))
    for i in data['consequents'].unique():
        print(i)

## Over

In [143]:
rules = pd.read_csv('RQ2\Over\ARM_consequents_0.5_0.9.csv')
rules = rules.dropna()
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Over'}),0.857675,1.0,0.857675,1.0,1.0,0.0,inf,0.0
1,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Over'}),0.774100,1.0,0.774100,1.0,1.0,0.0,inf,0.0
2,frozenset({'Skill use work - How often - Organ...,frozenset({'Qualification_Status_Over'}),0.581092,1.0,0.581092,1.0,1.0,0.0,inf,0.0
3,frozenset({'Skill use work - How often - Plann...,frozenset({'Qualification_Status_Over'}),0.545304,1.0,0.545304,1.0,1.0,0.0,inf,0.0
4,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Over'}),0.528134,1.0,0.528134,1.0,1.0,0.0,inf,0.0


In [144]:
print(rules.shape)
print(rules.dtypes)

(403204, 10)
antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
zhangs_metric         float64
dtype: object


In [145]:
rules_clean = pruneRules(rules)
rules_clean = cleanStrings(rules_clean)
rules_clean

(30, 10)
Range for LIFT:  1.0001865341172709  -  1.051924159103957
Range for CONFIDENCE:  0.9339764201500536  -  0.9829419583517944
Range for SUPPORT:  0.901323955316508  -  0.917873396772859


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,NATIVESPEAKER_Yes,"Qualification_Status_Over, Background - Born i...",0.934423,0.933802,0.917873,0.982289,1.051924,0.045307,3.737691,0.752718
1,Background - Born in country_Yes,"Qualification_Status_Over, NATIVESPEAKER_Yes",0.933802,0.934423,0.917873,0.982942,1.051924,0.045307,3.844355,0.745661
2,Background - Born in country_Yes,"Qualification_Status_Over, NATIVESPEAKER_Yes, ...",0.933802,0.934423,0.917873,0.982942,1.051924,0.045307,3.844355,0.745661
3,NATIVESPEAKER_Yes,"Qualification_Status_Over, Background - Born i...",0.934423,0.933802,0.917873,0.982289,1.051924,0.045307,3.737691,0.752718
4,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Over, NATIVESPEAKER_Yes",0.933802,0.934423,0.917873,0.982942,1.051924,0.045307,3.844355,0.745661
5,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Over, Background - Born i...",0.934423,0.933802,0.917873,0.982289,1.051924,0.045307,3.737691,0.752718
6,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Over, NATIVESPEAKER_Yes",0.978693,0.934423,0.916218,0.936166,1.001865,0.001706,1.027303,0.087374
7,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Over, Current work - Empl...",0.934423,0.978693,0.916218,0.980518,1.001865,0.001706,1.093699,0.028390
8,NATIVESPEAKER_Yes,"Qualification_Status_Over, Current work - Empl...",0.934423,0.978693,0.916218,0.980518,1.001865,0.001706,1.093699,0.028390
9,Current work - Employees working for you - Cou...,"Qualification_Status_Over, NATIVESPEAKER_Yes",0.978693,0.934423,0.916218,0.936166,1.001865,0.001706,1.027303,0.087374


In [146]:
printUnique(rules_clean)
rules_clean.to_csv('RQ2\Over_Rules.csv', index=False)

antecedents
8
NATIVESPEAKER_Yes
Background - Born in country_Yes
Skill use everyday life - ICT - Ever used computer_Yes, Background - Born in country_Yes
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes
Skill use everyday life - ICT - Ever used computer_Yes, Current work - Employees working for you - Count_1 to 10 people
Current work - Employees working for you - Count_1 to 10 people
Education - Formal qualification - Reason job related_Yes
Education - Formal qualification - Reason job related_Yes, Skill use everyday life - ICT - Ever used computer_Yes
------------------------------------------------------------------------------------------------------------------------------------------------------
consequents
9
Qualification_Status_Over, Background - Born in country_Yes
Qualification_Status_Over, NATIVESPEAKER_Yes
Qualification_Status_Over, NATIVESPEAKER_Yes, Skill use everyday life - ICT - Ever used computer_Yes
Qualification_Status_Over, Background - Born 

### Singular Cons

In [147]:
rules = pd.read_csv('RQ2\Over\ARM_consequents_ONE_0.5_0.9.csv')
rules = rules.dropna()
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Over'}),0.857675,1.0,0.857675,1.0,1.0,0.0,inf,0.0
1,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Over'}),0.774100,1.0,0.774100,1.0,1.0,0.0,inf,0.0
2,frozenset({'Skill use work - How often - Organ...,frozenset({'Qualification_Status_Over'}),0.581092,1.0,0.581092,1.0,1.0,0.0,inf,0.0
3,frozenset({'Skill use work - How often - Plann...,frozenset({'Qualification_Status_Over'}),0.545304,1.0,0.545304,1.0,1.0,0.0,inf,0.0
4,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Over'}),0.528134,1.0,0.528134,1.0,1.0,0.0,inf,0.0


In [148]:
print(rules.shape)
print(rules.dtypes)

(39419, 10)
antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
zhangs_metric         float64
dtype: object


In [149]:
rules_clean = pruneRules(rules, lift = 0)
rules_clean = cleanStrings(rules_clean)
rules_clean

(25, 10)
Range for LIFT:  1.0  -  1.0
Range for CONFIDENCE:  1.0  -  1.0
Range for SUPPORT:  0.901323955316508  -  1.0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,Current work - Employees working for you - Cou...,Qualification_Status_Over,0.978693,1.0,0.978693,1.0,1.0,0.0,inf,0.0
1,Education - Formal qualification - Reason job ...,Qualification_Status_Over,0.901324,1.0,0.901324,1.0,1.0,0.0,inf,0.0
2,Education - Formal qualification - Reason job ...,Qualification_Status_Over,0.901324,1.0,0.901324,1.0,1.0,0.0,inf,0.0
3,Skill use everyday life - ICT - Ever used comp...,Qualification_Status_Over,0.917873,1.0,0.917873,1.0,1.0,0.0,inf,0.0
4,Education - Formal qualification - Reason job ...,Qualification_Status_Over,0.944146,1.0,0.944146,1.0,1.0,0.0,inf,0.0
5,Skill use everyday life - ICT - Ever used comp...,Qualification_Status_Over,0.916218,1.0,0.916218,1.0,1.0,0.0,inf,0.0
6,Skill use everyday life - ICT - Ever used comp...,Qualification_Status_Over,0.915598,1.0,0.915598,1.0,1.0,0.0,inf,0.0
7,Education - Formal qualification - Reason job ...,Qualification_Status_Over,0.965039,1.0,0.965039,1.0,1.0,0.0,inf,0.0
8,Current status/work history - Current - Paid j...,Qualification_Status_Over,0.904427,1.0,0.904427,1.0,1.0,0.0,inf,0.0
9,Education - Formal qualification - Reason job ...,Qualification_Status_Over,0.902151,1.0,0.902151,1.0,1.0,0.0,inf,0.0


In [150]:
printUnique(rules_clean)
rules_clean.to_csv('RQ2\Over_Rules_ONE.csv', index=False)

antecedents
25
Current work - Employees working for you - Count_1 to 10 people
Education - Formal qualification - Reason job related_Yes, Background - Born in country_Yes
Education - Formal qualification - Reason job related_Yes, Background - Born in country_Yes, Skill use everyday life - ICT - Ever used computer_Yes
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes, Background - Born in country_Yes
Education - Formal qualification - Reason job related_Yes, Current work - Employees working for you - Count_1 to 10 people, Skill use everyday life - ICT - Ever used computer_Yes
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes, Current work - Employees working for you - Count_1 to 10 people
Skill use everyday life - ICT - Ever used computer_Yes, Background - Born in country_Yes, Current work - Employees working for you - Count_1 to 10 people
Education - Formal qualification - Reason job related_Yes, Skill use everyday life - ICT - Ever used 

## Under

In [151]:
rules = pd.read_csv('RQ2/Under/ARM_consequents_0.5_0.9.csv')
rules = rules.dropna()
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Under'}),0.829341,1.0,0.829341,1.0,1.0,0.0,inf,0.0
1,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Under'}),0.700599,1.0,0.700599,1.0,1.0,0.0,inf,0.0
2,frozenset({'Skill use work - How often - Organ...,frozenset({'Qualification_Status_Under'}),0.561677,1.0,0.561677,1.0,1.0,0.0,inf,0.0
3,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Under'}),0.533533,1.0,0.533533,1.0,1.0,0.0,inf,0.0
4,frozenset({'NATIVELANG_Test language same as n...,frozenset({'Qualification_Status_Under'}),0.716168,1.0,0.716168,1.0,1.0,0.0,inf,0.0


In [152]:
print(rules.shape)
print(rules.dtypes)

(638332, 10)
antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
zhangs_metric         float64
dtype: object


In [153]:
rules_clean = pruneRules(rules)
rules_clean = cleanStrings(rules_clean)
rules_clean

(54, 10)
Range for LIFT:  1.08224924657898  -  1.08224924657898
Range for CONFIDENCE:  0.983745123537061  -  0.9967061923583662
Range for SUPPORT:  0.9059880239520958  -  0.9059880239520958


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,NATIVESPEAKER_Yes,"Qualification_Status_Under, Background - Born ...",0.920958,0.908982,0.905988,0.983745,1.082249,0.068854,5.599425,0.961495
1,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Under, Current work - Emp...",0.908982,0.920958,0.905988,0.996706,1.082249,0.068854,23.997126,0.834983
2,NATIVESPEAKER_Yes,"Qualification_Status_Under, Current work - Emp...",0.920958,0.908982,0.905988,0.983745,1.082249,0.068854,5.599425,0.961495
3,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Under, NATIVESPEAKER_Yes",0.908982,0.920958,0.905988,0.996706,1.082249,0.068854,23.997126,0.834983
4,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Under, Background - Born ...",0.920958,0.908982,0.905988,0.983745,1.082249,0.068854,5.599425,0.961495
5,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Under, NATIVESPEAKER_Yes,...",0.908982,0.920958,0.905988,0.996706,1.082249,0.068854,23.997126,0.834983
6,Skill use everyday life - ICT - Ever used comp...,"Qualification_Status_Under, Background - Born ...",0.920958,0.908982,0.905988,0.983745,1.082249,0.068854,5.599425,0.961495
7,"Background - Born in country_Yes, Current work...","Qualification_Status_Under, Skill use everyday...",0.908982,0.920958,0.905988,0.996706,1.082249,0.068854,23.997126,0.834983
8,"NATIVESPEAKER_Yes, Current work - Employees wo...","Qualification_Status_Under, Skill use everyday...",0.920958,0.908982,0.905988,0.983745,1.082249,0.068854,5.599425,0.961495
9,Background - Born in country_Yes,"Qualification_Status_Under, Skill use everyday...",0.908982,0.920958,0.905988,0.996706,1.082249,0.068854,23.997126,0.834983


In [154]:
printUnique(rules_clean)
rules_clean.to_csv('RQ2/Under_Rules.csv', index=False)

antecedents
16
NATIVESPEAKER_Yes
Skill use everyday life - ICT - Ever used computer_Yes, Background - Born in country_Yes, Current work - Employees working for you - Count_1 to 10 people
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes, Current work - Employees working for you - Count_1 to 10 people
Skill use everyday life - ICT - Ever used computer_Yes, Background - Born in country_Yes
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes
Background - Born in country_Yes, Current work - Employees working for you - Count_1 to 10 people
NATIVESPEAKER_Yes, Current work - Employees working for you - Count_1 to 10 people
Background - Born in country_Yes
Current work - Employee or self-employed_Employee, Skill use everyday life - ICT - Ever used computer_Yes, Background - Born in country_Yes, Current work - Employees working for you - Count_1 to 10 people
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes, Current work - Em

### Singular Cons

In [155]:
rules = pd.read_csv('RQ2/Under/ARM_consequents_ONE_0.5_0.9.csv')
rules = rules.dropna()
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Under'}),0.829341,1.0,0.829341,1.0,1.0,0.0,inf,0.0
1,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Under'}),0.700599,1.0,0.700599,1.0,1.0,0.0,inf,0.0
2,frozenset({'Skill use work - How often - Organ...,frozenset({'Qualification_Status_Under'}),0.561677,1.0,0.561677,1.0,1.0,0.0,inf,0.0
3,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Under'}),0.533533,1.0,0.533533,1.0,1.0,0.0,inf,0.0
4,frozenset({'NATIVELANG_Test language same as n...,frozenset({'Qualification_Status_Under'}),0.716168,1.0,0.716168,1.0,1.0,0.0,inf,0.0


In [156]:
print(rules.shape)
print(rules.dtypes)

(50167, 10)
antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
zhangs_metric         float64
dtype: object


In [157]:
rules_clean = pruneRules(rules, lift = 0)
rules_clean = cleanStrings(rules_clean)
rules_clean

(55, 10)
Range for LIFT:  1.0  -  1.0
Range for CONFIDENCE:  1.0  -  1.0
Range for SUPPORT:  0.9005988023952096  -  1.0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,Current work - Employee or self-employed_Employee,Qualification_Status_Under,1.000000,1.0,1.000000,1.0,1.0,0.0,inf,0.0
1,Skill use everyday life - ICT - Ever used comp...,Qualification_Status_Under,0.920958,1.0,0.920958,1.0,1.0,0.0,inf,0.0
2,Skill use everyday life - ICT - Ever used comp...,Qualification_Status_Under,1.000000,1.0,1.000000,1.0,1.0,0.0,inf,0.0
3,Education - Formal qualification - Reason job ...,Qualification_Status_Under,0.960479,1.0,0.960479,1.0,1.0,0.0,inf,0.0
4,Skill use everyday life - ICT - Experience wit...,Qualification_Status_Under,0.900599,1.0,0.900599,1.0,1.0,0.0,inf,0.0
5,Current work - Employee or self-employed_Emplo...,Qualification_Status_Under,0.905988,1.0,0.905988,1.0,1.0,0.0,inf,0.0
6,Current work - Employee or self-employed_Emplo...,Qualification_Status_Under,0.908982,1.0,0.908982,1.0,1.0,0.0,inf,0.0
7,Skill use everyday life - ICT - Ever used comp...,Qualification_Status_Under,0.920958,1.0,0.920958,1.0,1.0,0.0,inf,0.0
8,Skill use everyday life - ICT - Ever used comp...,Qualification_Status_Under,0.928743,1.0,0.928743,1.0,1.0,0.0,inf,0.0
9,Education - Formal qualification - Reason job ...,Qualification_Status_Under,0.960479,1.0,0.960479,1.0,1.0,0.0,inf,0.0


In [158]:
printUnique(rules_clean)
rules_clean.to_csv('RQ2/Under_Rules_ONE.csv', index=False)

antecedents
55
Current work - Employee or self-employed_Employee
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes, Current work - Employees working for you - Count_1 to 10 people
Skill use everyday life - ICT - Ever used computer_Yes, Current work - Employee or self-employed_Employee, Current work - Employees working for you - Count_1 to 10 people
Education - Formal qualification - Reason job related_Yes, Current work - Employee or self-employed_Employee, Current work - Employees working for you - Count_1 to 10 people
Skill use everyday life - ICT - Experience with computer everyday life_Yes, Current work - Employee or self-employed_Employee, Current work - Employees working for you - Count_1 to 10 people
Current work - Employee or self-employed_Employee, NATIVESPEAKER_Yes, Background - Born in country_Yes
Current work - Employee or self-employed_Employee, Background - Born in country_Yes, Skill use everyday life - ICT - Ever used computer_Yes
Skill use everyda

## Equal

In [159]:
rules = pd.read_csv('RQ2\Equal\ARM_consequents_0.5_0.9.csv')
rules = rules.dropna()
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Equal'}),0.812928,1.0,0.812928,1.0,1.0,0.0,inf,0.0
1,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Equal'}),0.663512,1.0,0.663512,1.0,1.0,0.0,inf,0.0
2,frozenset({'Skill use work - How often - Organ...,frozenset({'Qualification_Status_Equal'}),0.590012,1.0,0.590012,1.0,1.0,0.0,inf,0.0
3,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Equal'}),0.530810,1.0,0.530810,1.0,1.0,0.0,inf,0.0
4,frozenset({'NATIVELANG_Test language same as n...,frozenset({'Qualification_Status_Equal'}),0.740435,1.0,0.740435,1.0,1.0,0.0,inf,0.0


In [160]:
print(rules.shape)
print(rules.dtypes)

(797219, 10)
antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
zhangs_metric         float64
dtype: object


In [161]:
rules_clean = pruneRules(rules)
rules_clean = cleanStrings(rules_clean)
rules_clean

(108, 10)
Range for LIFT:  1.0023796923457946  -  1.0746374096900175
Range for CONFIDENCE:  0.9503027771977448  -  0.9863466196872936
Range for SUPPORT:  0.9019331453886428  -  0.9164317358034636


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,NATIVESPEAKER_Yes,"Qualification_Status_Equal, Background - Born ...",0.917841,0.914418,0.901933,0.982668,1.074637,0.062642,4.937754,0.845359
1,Current work - Employee or self-employed_Emplo...,"Qualification_Status_Equal, Skill use everyday...",0.914418,0.917841,0.901933,0.986347,1.074637,0.062642,6.017461,0.811545
2,"Background - Born in country_Yes, Current work...","Qualification_Status_Equal, Current work - Emp...",0.914418,0.917841,0.901933,0.986347,1.074637,0.062642,6.017461,0.811545
3,Current work - Employee or self-employed_Emplo...,"Qualification_Status_Equal, NATIVESPEAKER_Yes,...",0.914418,0.917841,0.901933,0.986347,1.074637,0.062642,6.017461,0.811545
4,"NATIVESPEAKER_Yes, Current work - Employees wo...","Qualification_Status_Equal, Current work - Emp...",0.917841,0.914418,0.901933,0.982668,1.074637,0.062642,4.937754,0.845359
...,...,...,...,...,...,...,...,...,...,...
103,Current status/work history - Current - Paid j...,"Qualification_Status_Equal, Education - Formal...",0.948047,0.964358,0.916432,0.966653,1.002380,0.002176,1.068817,0.045696
104,Education - Formal qualification - Reason job ...,"Qualification_Status_Equal, Current status/wor...",0.964358,0.948047,0.916432,0.950303,1.002380,0.002176,1.045396,0.066607
105,Current status/work history - Current - Paid j...,"Qualification_Status_Equal, Education - Formal...",0.948047,0.964358,0.916432,0.966653,1.002380,0.002176,1.068817,0.045696
106,Current status/work history - Current - Paid j...,"Qualification_Status_Equal, Education - Formal...",0.948047,0.964358,0.916432,0.966653,1.002380,0.002176,1.068817,0.045696


In [162]:
printUnique(rules_clean)
rules_clean.to_csv('RQ2\Equal_Rules.csv', index=False)

antecedents
34
NATIVESPEAKER_Yes
Current work - Employee or self-employed_Employee, Background - Born in country_Yes
Background - Born in country_Yes, Current work - Employees working for you - Count_1 to 10 people
NATIVESPEAKER_Yes, Current work - Employees working for you - Count_1 to 10 people
Current work - Employee or self-employed_Employee, NATIVESPEAKER_Yes
Background - Born in country_Yes
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes, Current work - Employee or self-employed_Employee
Current work - Employee or self-employed_Employee, Background - Born in country_Yes, Current work - Employees working for you - Count_1 to 10 people
Skill use everyday life - ICT - Ever used computer_Yes, NATIVESPEAKER_Yes
Current work - Employee or self-employed_Employee, Skill use everyday life - ICT - Ever used computer_Yes, Background - Born in country_Yes
Skill use everyday life - ICT - Ever used computer_Yes, Background - Born in country_Yes
Current work - Employee

### Singular Cons

In [163]:
rules = pd.read_csv('RQ2\Equal\ARM_consequents_ONE_0.5_0.9.csv')
rules = rules.dropna()
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Equal'}),0.812928,1.0,0.812928,1.0,1.0,0.0,inf,0.0
1,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Equal'}),0.663512,1.0,0.663512,1.0,1.0,0.0,inf,0.0
2,frozenset({'Skill use work - How often - Organ...,frozenset({'Qualification_Status_Equal'}),0.590012,1.0,0.590012,1.0,1.0,0.0,inf,0.0
3,frozenset({'Skill use work - ICT - Computer - ...,frozenset({'Qualification_Status_Equal'}),0.530810,1.0,0.530810,1.0,1.0,0.0,inf,0.0
4,frozenset({'NATIVELANG_Test language same as n...,frozenset({'Qualification_Status_Equal'}),0.740435,1.0,0.740435,1.0,1.0,0.0,inf,0.0


In [164]:
print(rules.shape)
print(rules.dtypes)

(56927, 10)
antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
zhangs_metric         float64
dtype: object


In [165]:
rules_clean = pruneRules(rules, lift = 0)
rules_clean = cleanStrings(rules_clean)
rules_clean

(63, 10)
Range for LIFT:  1.0  -  1.0
Range for CONFIDENCE:  1.0  -  1.0
Range for SUPPORT:  0.900926298832058  -  1.0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,Current work - Employee or self-employed_Employee,Qualification_Status_Equal,1.000000,1.0,1.000000,1.0,1.0,0.0,inf,0.0
1,Skill use everyday life - ICT - Experience wit...,Qualification_Status_Equal,0.900926,1.0,0.900926,1.0,1.0,0.0,inf,0.0
2,Current work - Employee or self-employed_Emplo...,Qualification_Status_Equal,0.901933,1.0,0.901933,1.0,1.0,0.0,inf,0.0
3,Skill use everyday life - ICT - Ever used comp...,Qualification_Status_Equal,0.914418,1.0,0.914418,1.0,1.0,0.0,inf,0.0
4,Current work - Employee or self-employed_Emplo...,Qualification_Status_Equal,0.917841,1.0,0.917841,1.0,1.0,0.0,inf,0.0
...,...,...,...,...,...,...,...,...,...,...
58,Education - Formal qualification - Reason job ...,Qualification_Status_Equal,0.964358,1.0,0.964358,1.0,1.0,0.0,inf,0.0
59,Skill use everyday life - ICT - Experience wit...,Qualification_Status_Equal,0.900926,1.0,0.900926,1.0,1.0,0.0,inf,0.0
60,Current work - Employee or self-employed_Emplo...,Qualification_Status_Equal,0.914418,1.0,0.914418,1.0,1.0,0.0,inf,0.0
61,Current work - Employee or self-employed_Emplo...,Qualification_Status_Equal,0.917841,1.0,0.917841,1.0,1.0,0.0,inf,0.0


In [166]:
printUnique(rules_clean)
rules_clean.to_csv('RQ2\Equal_Rules_ONE.csv', index=False)

antecedents
63
Current work - Employee or self-employed_Employee
Skill use everyday life - ICT - Experience with computer everyday life_Yes, Skill use everyday life - ICT - Ever used computer_Yes, Current work - Employees working for you - Count_1 to 10 people
Current work - Employee or self-employed_Employee, NATIVESPEAKER_Yes, Background - Born in country_Yes
Skill use everyday life - ICT - Ever used computer_Yes, Current work - Employee or self-employed_Employee, Background - Born in country_Yes
Current work - Employee or self-employed_Employee, NATIVESPEAKER_Yes, Skill use everyday life - ICT - Ever used computer_Yes
Current status/work history - Current - Paid job or family business (DERIVED BY CAPI)_Yes, paid work one job or business, Current work - Employee or self-employed_Employee, Skill use everyday life - ICT - Ever used computer_Yes
Education - Formal qualification - Reason job related_Yes, Current status/work history - Current - Paid job or family business (DERIVED BY CAPI